<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# AI-First Finance

**OLS Regression as the Benchmark**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

### The use of the "Python 3.10, Numpy 1.26.4"  kernel is recommended.

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
# !pip install cufflinks

In [ ]:
import math
import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
import cufflinks
cufflinks.set_config_file(offline=True)

## OLS Regression

### Simple Regression

In [ ]:
x = np.linspace(0, 5, 100)
y = 2 + 0.5 * x + np.random.standard_normal(len(x)) * 0.15

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro');

In [ ]:
np.cov(x, y)

In [ ]:
cov = np.cov(x, y)[0, 1]

In [ ]:
beta = cov / x.var()

In [ ]:
alpha = y.mean() - beta * x.mean()

In [ ]:
alpha, beta

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, alpha + beta * x, lw=2.0)
plt.plot(x, y, 'ro');

In [ ]:
r = alpha + beta * x - y

Check for assumptions:
* **linearity**: given
* **independence**: given
* **zero mean**: mean zero given (see below)
* **no correlation**: correlation of zero (see below)
* **homoscedasticity**: given (see below)
* **no autocorrelation**: given (see below)

See also [Linear Regression Assumptions](https://medium.com/datadriveninvestor/linear-regression-assumptions-f2252b8e2912) or [Ordinary Least Squares](https://en.wikipedia.org/wiki/Ordinary_least_squares).

In [ ]:
r.mean()

In [ ]:
np.corrcoef(r, x)

In [ ]:
from scipy.stats import bartlett

In [ ]:
split = int(len(x) / 2)

In [ ]:
bartlett(r[:split], r[split:])

In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_acf(r, ax=ax);

### Monomial Regression

In [ ]:
x = np.linspace(0, 5, 100)
y = 2 - 0.5 * x ** 2 + 0.1 * x ** 3 + np.random.standard_normal(len(x)) * 0.15

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, y, 'ro');

In [ ]:
reg = np.polyfit(x, y, deg=3)
reg

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, np.polyval(reg, x), lw=2.0)
plt.plot(x, y, 'ro');

In [ ]:
r = np.polyval(reg, x) - y

Check for assumptions:
* **linearity**: given
* **independence**: given
* **zero mean**: given
* **no correlation**: given
* **homoscedasticity**: given
* **no autocorrelation**: given

In [ ]:
r.mean()

In [ ]:
np.corrcoef(r, x)

In [ ]:
split = int(len(x) / 2)

In [ ]:
bartlett(r[:split], r[split:])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_acf(r, ax=ax);

### Multivariate Regression

In [ ]:
x1 = np.random.random(200) * 5
x2 = np.random.random(200) * 5
y = 2 - 0.5 * x1 ** 2 + 0.1 * x2 ** 3 + np.random.standard_normal(len(x1)) * 0.15

In [ ]:
df = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y, 'text': '', 'categories': ''})

In [ ]:
df.iplot(kind='scatter3d', x='x1', y='x2', z='y',
         text='text', categories='categories', size=4)

In [ ]:
B = np.array((x1, x2)).T

In [ ]:
B[:5]

In [ ]:
reg = np.linalg.lstsq(B, y, rcond=-1)

In [ ]:
reg
# 1 optimal regression parameters
# 2 squared Euclidean norm for residuals
# 3 rank of the matrix
# 4 singular values of matrix

In [ ]:
y_ = np.dot(B, reg[0])

In [ ]:
r = y_ - y

In [ ]:
df['r'] = r

Check for assumptions:
* **linearity**: given
* **independence**: given (see corr x1 & x2)
* **zero mean**: non-zero mean
* **no correlation**: not given
* **homoscedasticity**: given
* **no autocorrelation**: given

In [ ]:
df.corr(numeric_only=True)

In [ ]:
r.mean()

In [ ]:
split = int(len(x) / 2)

In [ ]:
# help(bartlett)

In [ ]:
bartlett(r[:split], r[split:])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_acf(r, ax=ax);

## Fast and Scalable

In [ ]:
%%time
N = 10000000
x1 = np.random.random(N) * 5
x2 = np.random.random(N) * 5
x3 = np.random.random(N) * 5
B = np.array((x1, x2, x3)).T
y = 2 + 5 * x1 - 0.5 * x2 ** 2 + 0.1 * x3 ** 3 + np.random.standard_normal(len(x1)) * 0.15

In [ ]:
x1.nbytes * 4 / 1e6  # MB

In [ ]:
%time np.linalg.lstsq(B, y, rcond=-1)[0]

## Linear Regression with Scikit-Learn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
x = np.linspace(0, 5, 50)
y = 2 - 0.5 * x ** 2 + 0.1 * x ** 3 + np.random.standard_normal(len(x)) * 0.1

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(np.atleast_2d(x).T, y)

In [ ]:
pred = model.predict(np.atleast_2d(x).T)
pred[:5]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x, pred)
plt.plot(x, y, 'ro');

In [ ]:
r2_score(y, [y.mean()] * len(y))

In [ ]:
r2_score(y, pred)

## Monomial Regression with Scikit-Learn

See VanderPlas: Python Data Science Handbook (O'Reilly).

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

In [ ]:
x = np.atleast_2d(x).T

In [ ]:
x_ = np.atleast_2d(np.linspace(x.min(), x.max(), 50)).T

In [ ]:
res = {}
for deg in [1, 3, 5]:
    model = PolynomialRegression(deg)
    model.fit(x, y)
    res[deg] = model.predict(x_)

In [ ]:
plt.figure(figsize=(10, 6))
for key in res:
    plt.plot(x_, res[key], label=f'degree={key}')
plt.plot(x, y, 'ro')
plt.legend();

In [ ]:
for deg in [1, 2, 3, 4, 5]:
    y_ = PolynomialRegression(deg).fit(x.reshape(-1, 1), y).predict(x.reshape(-1, 1))
    R2 = r2_score(y, y_)
    print(f'degree={deg} | R2={R2}')

## Model Validation

### Simple Split

In [ ]:
split = int(len(x) * 0.6)

In [ ]:
model = PolynomialRegression(2)

In [ ]:
model.fit(x[:split], y[:split])

In [ ]:
pred = model.predict(x)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x[:split], pred[:split], 'b--')
plt.plot(x[split:], pred[split:], 'y--')
plt.plot(x, y, 'ro');

### Random Split

In [ ]:
i = np.arange(len(x))
np.random.shuffle(i)

In [ ]:
model.fit(x[i[:split]], y[i[:split]])

In [ ]:
pred = model.predict(x)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(x[:split], pred[:split], 'b--')
plt.plot(x[split:], pred[split:], 'y--')
plt.plot(x, y, 'ro');

## Validation Curves

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
degrees = np.arange(1, 6)

In [ ]:
# validation_curve?

In [ ]:
train_score, val_score = validation_curve(PolynomialRegression(), X=x, y=y,
                                         param_name='polynomialfeatures__degree',
                                         param_range=degrees, cv=5)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(degrees, np.median(train_score, axis=1), label='training score')
plt.plot(degrees, np.median(val_score, axis=1), label='validation score')
plt.legend();

## Random Walks

Eugene F. Fama (1965): “Random Walks in Stock Market Prices”:

> “For many years, economists, statisticians, and teachers of finance have been interested in developing and testing models of stock price behavior. One important model that has evolved from this research is the theory of random walks. This theory casts serious doubt on many other methods for describing and predicting stock price behavior—methods that have considerable popularity outside the academic world. For example, we shall see later that, if the random-walk theory is an accurate description of reality, then the various “technical” or “chartist” procedures for predicting stock prices are completely without value.”

Michael Jensen (1978): “Some Anomalous Evidence Regarding Market Efficiency”:

>“A market is efficient with respect to an information set S if it is impossible to make economic profits by trading on the basis of information set S.”

If a stock price follows a (simple) random walk (no drift & normally distributed returns), then it rises and falls with the same probability of 50% (“toss of a coin”).

**In such a case, the best predictor of tomorrow’s stock price — in a least-squares sense — is today’s stock price.**

### Retrieving Cross-Asset Data

In [ ]:
url = 'http://hilpisch.com/tr_eikon_eod_data.csv'

In [ ]:
data = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [ ]:
data.head()

In [ ]:
data.info()

### Calculating the Log Returns

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
rets.head()

### Plotting the Data

In [ ]:
data.normalize().iplot(kind='lines')

In [ ]:
rets.iplot(kind='histogram', subplots=True)

In [ ]:
rets.corr().iplot(kind='heatmap', colorscale='blues')

### Preparing Lagged Data

In [ ]:
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(data[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        df[col] = df[ric].shift(lag)  # creates the lagged data column
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

In [ ]:
lags = 7  # seven historical lags

In [ ]:
dfs = {}
for sym in data.columns:
    df, cols = add_lags(data, sym, lags)
    dfs[sym] = df

In [ ]:
cols  # the column names for the lags

In [ ]:
dfs.keys()  # the keys of the dictonary

In [ ]:
dfs['AAPL.O'].head(7)

### Implementing OLS Regression

In [ ]:
regs = {}
for sym in data.columns:
    df = dfs[sym]  # getting data for the RIC
    reg = np.linalg.lstsq(df[cols], df[sym], rcond=-1)[0]  # the OLS regression
    regs[sym] = reg  # storing the results

In [ ]:
for sym in data.columns:
    print('{:10} | {}'.format(sym, regs[sym].round(4)))

In [ ]:
rega = np.stack(list(regs.values()))  # combines the regression results

In [ ]:
rega.mean(axis=0)  # mean values by column

In [ ]:
regd = pd.DataFrame(rega, columns=cols, index=data.columns)  # converting the results to DataFrame

In [ ]:
regd

In [ ]:
regd.iplot(kind='bar')

In [ ]:
regd.mean().iplot(kind='bar')

## Cross Check

In [ ]:
sym = 'AAPL.O'

In [ ]:
reg = regd.loc[sym].values

In [ ]:
y_ = np.dot(dfs[sym][cols], reg)

In [ ]:
r = y_ - dfs[sym][sym]

Check for assumptions:
* **linearity**: given
* **independence**: <b style="color: red;">not at all</b>
* **zero mean**: somehow
* **no correlation**: given
* **homoscedasticity**: **not given**
* **no autocorrelation**: given
* **stationarity**: **not given**

In [ ]:
dfs[sym][cols].corr()

In [ ]:
r.mean()

In [ ]:
split = int(len(x) / 2)

In [ ]:
np.corrcoef(r, dfs[sym]['lag_3'])

In [ ]:
bartlett(r[:split], r[split:])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_acf(r, ax=ax);

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
adfuller(dfs[sym][sym])

## A Gleam of Hope?

### Multi Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(250, 250), max_iter=100)

In [ ]:
sym = 'AAPL.O'

### Return Patterns

In [ ]:
rets = np.log(dfs[sym] / dfs[sym].shift(1)).dropna()

In [ ]:
adfuller(rets[sym])

In [ ]:
rets['d'] = np.sign(rets[sym])

In [ ]:
# rets[cols] = np.sign(rets[cols])

In [ ]:
# rets[cols] -= rets[cols].mean()
# rets[cols] /= rets[cols].std()

In [ ]:
rets.head()

### Model Fitting & Prediction

In [ ]:
model = model.fit(rets[cols], rets['d'])

In [ ]:
rets['p'] = model.predict(rets[cols])

In [ ]:
accuracy_score(rets['d'], rets['p'])

### Vectorized Backtesting

In [ ]:
(rets['p'].diff() != 0).sum()

In [ ]:
rets['strategy'] = rets['p'] * rets[sym]

In [ ]:
rets[[sym, 'strategy']].cumsum().apply(np.exp).iplot()

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">